In [48]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [49]:
#leer la bd
df = pd.read_csv("../../data/processed/4.4/tmp/BD_ID_CLIENTE_N.POLIZA_X_MES_actual.csv", index_col="Unnamed: 0")
df.head()

,2004-02,2004-03,2004-04,2004-05,2004-06,2004-07,2004-08,2004-09,2004-10,2004-11,...,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12
1690,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0
7968,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9860,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN


In [50]:
#agrego la bd de datos cliente
df_cli = pd.read_csv("../../data/processed/BD_ID_CLIENTE_D.Pers_Modo.Pago.csv", index_col="Unnamed: 0")
df_cli.set_index("CUSTOMER_ID", inplace=True)
df_cli.sort_index(inplace=True)
df_cli.head()

,DATE_OF_BIRTH,GENDER,MARITAL_STATUS,EDUCATION,EDUCATION_LEVEL,OCCUPATION,CITY,LOCALITY,FECHA_PROCESO,edad,ModoPago,edad_r
CUSTOMER_ID,,,,,,,,,,,,
1690,1943-03-06,M,Casado,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0
5575,1966-06-10,M,Casado,NaN,NaN,ESCB,CAPITAL FEDERAL,CAPITAL FEDERAL,31/01/2017,51.0,Mix,4.0
7968,1911-01-11,M,Casado,NaN,NaN,NaN,NaN,NaN,31/01/2018,107.0,CC,0.0
9028,1961-07-15,M,Casado,NaN,NaN,ARQU,CAPITAL FEDERAL,CAPITAL FEDERAL,31/01/2018,56.0,Mix,4.0
9860,1967-04-06,M,Soltero,NaN,NaN,EMPL,CAPITAL FEDERAL,CAPITAL FEDERAL,31/01/2016,50.0,Mix,4.0


In [51]:
df_canc = pd.read_csv("../../data/processed/4.4/tmp/mes actual - BD_ID_CLIENTE_N.CANCELACIONES_X_MES.csv", index_col="Unnamed: 0")
df_canc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41754 entries, 1690 to 21260067
Columns: 166 entries, 2004-03 to 2017-12
dtypes: float64(166)
memory usage: 53.2 MB


In [52]:
df_rel = pd.read_csv("../../data/processed/4.4/tmp/BD_ID_CLIENTE_N.INTERACCIONES-GENERALES_X_MES.csv", index_col="Unnamed: 0")
df_rel.sort_index(inplace=True)
df_rel=df_rel.iloc[:,:-1]
df_rel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48704 entries, 1690 to 21179943
Data columns (total 60 columns):
2013-01    8305 non-null float64
2013-02    7536 non-null float64
2013-03    8435 non-null float64
2013-04    8851 non-null float64
2013-05    9467 non-null float64
2013-06    8731 non-null float64
2013-07    9879 non-null float64
2013-08    9599 non-null float64
2013-09    9181 non-null float64
2013-10    11058 non-null float64
2013-11    11214 non-null float64
2013-12    12913 non-null float64
2014-01    14679 non-null float64
2014-02    13908 non-null float64
2014-03    14773 non-null float64
2014-04    16164 non-null float64
2014-05    17499 non-null float64
2014-06    16654 non-null float64
2014-07    16489 non-null float64
2014-08    16073 non-null float64
2014-09    18184 non-null float64
2014-10    17242 non-null float64
2014-11    16492 non-null float64
2014-12    18412 non-null float64
2015-01    19964 non-null float64
2015-02    18099 non-null float64
2015-03   

# armando bloques 9 m (6 train, 3 target): desde 2014

### Creo lista de dataframes

In [53]:
df = df.iloc[:,:-1]
df_canc = df_canc.iloc[:,:-1] #para definir churn
df_rel = df_rel.iloc[:,:-1]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 1690 to 21260067
Columns: 166 entries, 2004-02 to 2017-11
dtypes: float64(166)
memory usage: 63.7 MB


In [54]:
df.head()

,2004-02,2004-03,2004-04,2004-05,2004-06,2004-07,2004-08,2004-09,2004-10,2004-11,...,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11
1690,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0
7968,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9860,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN


In [55]:
df_canc.head()

,2004-03,2004-04,2004-05,2004-06,2004-07,2004-08,2004-09,2004-10,2004-11,2004-12,...,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11
1690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
7968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [56]:
df_rel.head()

,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,...,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11
1690,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5575,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,2.0,3.0,...,7.0,NaN,2.0,NaN,1.0,NaN,7.0,19.0,2.0,3.0
7968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9028,4.0,5.0,1.0,1.0,3.0,3.0,1.0,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,6.0,4.0,2.0,6.0,3.0
9860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,12.0,2.0,NaN,NaN,NaN,NaN,9.0,3.0,NaN


In [57]:
#ANALIZO DESDE 2014
df = df.loc[:,"2013-01":]
df_canc = df_canc.loc[:,"2013-01":]
df_rel = df_rel.loc[:,"2013-01":]

In [58]:
len(df.columns)

59

In [59]:
for i in range( (len(df.columns)-1) ,8, -1):
    print("{}-{}".format(df.columns[i-9],df.columns[i-3]))

2017-02-2017-08
2017-01-2017-07
2016-12-2017-06
2016-11-2017-05
2016-10-2017-04
2016-09-2017-03
2016-08-2017-02
2016-07-2017-01
2016-06-2016-12
2016-05-2016-11
2016-04-2016-10
2016-03-2016-09
2016-02-2016-08
2016-01-2016-07
2015-12-2016-06
2015-11-2016-05
2015-10-2016-04
2015-09-2016-03
2015-08-2016-02
2015-07-2016-01
2015-06-2015-12
2015-05-2015-11
2015-04-2015-10
2015-03-2015-09
2015-02-2015-08
2015-01-2015-07
2014-12-2015-06
2014-11-2015-05
2014-10-2015-04
2014-09-2015-03
2014-08-2015-02
2014-07-2015-01
2014-06-2014-12
2014-05-2014-11
2014-04-2014-10
2014-03-2014-09
2014-02-2014-08
2014-01-2014-07
2013-12-2014-06
2013-11-2014-05
2013-10-2014-04
2013-09-2014-03
2013-08-2014-02
2013-07-2014-01
2013-06-2013-12
2013-05-2013-11
2013-04-2013-10
2013-03-2013-09
2013-02-2013-08
2013-01-2013-07


In [60]:
#termino en 5 para que no de error en el rango de las columnas
list_df = list()
list_canc = list()
list_rel = list()
for i in range( (len(df.columns)) ,8, -1):
    #print("{}-{}".format(i-9,i))
    list_df.append(df.iloc[:,i-9:i])
    list_rel.append(df_rel.iloc[:,i-9:i-3])
    list_canc.append(df_canc.iloc[:,i-3:i])
len(list_df)

51

In [61]:
list_df[0]

,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11
1690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5575,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0
7968,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9028,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9860,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
11820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
14923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16007,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
16318,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
17153,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [62]:
list_df[(len(list_df)-1)]

,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09
1690,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5575,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7968,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9028,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11820,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0
14923,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0
16007,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
16318,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
17153,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [63]:
tmp = list()
for i in range(0, len(list_df)):
    tmp.append(pd.merge(list_df[i], list_rel[i], left_index=True, right_index=True, suffixes=["","_rel"]))
    tmp[i] = pd.merge(tmp[i], list_canc[i], left_index=True, right_index=True, suffixes=["","_canc"])
#    tmp[i] = pd.merge(tmp[i], list_df_dif[i], left_index=True, right_index=True, suffixes=["","_did"])
len(tmp)

51

In [64]:
tmp[0].head()

,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-03_rel,2017-04_rel,2017-05_rel,2017-06_rel,2017-07_rel,2017-08_rel,2017-09_canc,2017-10_canc,2017-11_canc
1690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5575,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,NaN,2.0,NaN,1.0,NaN,7.0,2.0,NaN,NaN
7968,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9028,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,6.0,4.0,NaN,NaN,NaN
9860,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,12.0,2.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [65]:
tmp[len(tmp)-1].head()

,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-01_rel,2013-02_rel,2013-03_rel,2013-04_rel,2013-05_rel,2013-06_rel,2013-07_canc,2013-08_canc,2013-09_canc
1690,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
5575,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
7968,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9028,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,1.0,1.0,3.0,3.0,NaN,NaN,NaN
9860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Renombre las columnas de cada dataframe

In [66]:
print(tmp[0].columns)
len(tmp[0].columns)

Index(['2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08',
       '2017-09', '2017-10', '2017-11', '2017-03_rel', '2017-04_rel',
       '2017-05_rel', '2017-06_rel', '2017-07_rel', '2017-08_rel',
       '2017-09_canc', '2017-10_canc', '2017-11_canc'],
      dtype='object')


18

In [67]:
#cambiando el nombre de las fechas a versión generica ( 6 meses train, 3 pred)
cols = ["6m", "5m", "4m", "3m", "2m", "1m", "3p", "2p", "1p",
       '6m_rel', '5m_rel', '4m_rel', '3m_rel', '2m_rel', '1m_rel',
       '3m_canc', '2m_canc', '1m_canc']
len(cols)

18

In [68]:
#cambiando el nombre de las fechas a versión generica ( 6 meses train, 3 pred)
for i in range(0, len(tmp)):
    tmp[i].rename(columns=dict(zip(tmp[i].columns, cols)),inplace=True)

### Integrando lista de dataframes en un único dataframe

In [69]:
#contar cantidad total de registros
tot = 0
for i in range(0, len(tmp)):
    tot = tot + len(tmp[i])
    
print(tot)

2086257


In [70]:
bd = pd.concat(tmp)
bd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2086257 entries, 1690 to 21096072
Data columns (total 18 columns):
6m         float64
5m         float64
4m         float64
3m         float64
2m         float64
1m         float64
3p         float64
2p         float64
1p         float64
6m_rel     float64
5m_rel     float64
4m_rel     float64
3m_rel     float64
2m_rel     float64
1m_rel     float64
3m_canc    float64
2m_canc    float64
1m_canc    float64
dtypes: float64(18)
memory usage: 302.4 MB


### Integrando con dataframe clientes

In [71]:
df_int = pd.merge(bd, df_cli, left_index=True, right_index=True)
df_int.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2086257 entries, 1690 to 21096072
Data columns (total 30 columns):
6m                 float64
5m                 float64
4m                 float64
3m                 float64
2m                 float64
1m                 float64
3p                 float64
2p                 float64
1p                 float64
6m_rel             float64
5m_rel             float64
4m_rel             float64
3m_rel             float64
2m_rel             float64
1m_rel             float64
3m_canc            float64
2m_canc            float64
1m_canc            float64
DATE_OF_BIRTH      object
GENDER             object
MARITAL_STATUS     object
EDUCATION          object
EDUCATION_LEVEL    object
OCCUPATION         object
CITY               object
LOCALITY           object
FECHA_PROCESO      object
edad               float64
ModoPago           object
edad_r             float64
dtypes: float64(20), object(10)
memory usage: 493.4+ MB


In [72]:
df_int["id"] = df_int.index
df_int[df_int.duplicated()]

,6m,5m,4m,3m,2m,1m,3p,2p,1p,6m_rel,...,EDUCATION,EDUCATION_LEVEL,OCCUPATION,CITY,LOCALITY,FECHA_PROCESO,edad,ModoPago,edad_r,id
1690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0,1690
1690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0,1690
1690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0,1690
1690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0,1690
1690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0,1690
1690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0,1690
1690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0,1690
1690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0,1690
1690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0,1690
1690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0,1690


In [73]:
df_int.drop_duplicates(inplace=True)
df_int = df_int.iloc[:,:-1] #elimino la columna id
df_int.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1390090 entries, 1690 to 21096072
Data columns (total 30 columns):
6m                 1160726 non-null float64
5m                 1163250 non-null float64
4m                 1164737 non-null float64
3m                 1165245 non-null float64
2m                 1164501 non-null float64
1m                 1162572 non-null float64
3p                 1159350 non-null float64
2p                 1154840 non-null float64
1p                 1149201 non-null float64
6m_rel             739916 non-null float64
5m_rel             751875 non-null float64
4m_rel             763123 non-null float64
3m_rel             773087 non-null float64
2m_rel             782870 non-null float64
1m_rel             791270 non-null float64
3m_canc            52942 non-null float64
2m_canc            53402 non-null float64
1m_canc            53837 non-null float64
DATE_OF_BIRTH      1379442 non-null object
GENDER             1389564 non-null object
MARITAL_STATUS   

In [74]:
#variables descartadas
delete_cols = ["DATE_OF_BIRTH", "FECHA_PROCESO", "OCCUPATION"]#, "LOCALITY", "edad"]
df_fin = df_int.drop(delete_cols, axis=1)
df_fin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1390090 entries, 1690 to 21096072
Data columns (total 27 columns):
6m                 1160726 non-null float64
5m                 1163250 non-null float64
4m                 1164737 non-null float64
3m                 1165245 non-null float64
2m                 1164501 non-null float64
1m                 1162572 non-null float64
3p                 1159350 non-null float64
2p                 1154840 non-null float64
1p                 1149201 non-null float64
6m_rel             739916 non-null float64
5m_rel             751875 non-null float64
4m_rel             763123 non-null float64
3m_rel             773087 non-null float64
2m_rel             782870 non-null float64
1m_rel             791270 non-null float64
3m_canc            52942 non-null float64
2m_canc            53402 non-null float64
1m_canc            53837 non-null float64
GENDER             1389564 non-null object
MARITAL_STATUS     1281584 non-null object
EDUCATION        

In [75]:
#elimino los registros todos vacios
#elimino aquellos registros que ya se que no son más clientes
df_fin = df_fin[df_fin.iloc[:,:6].sum(axis=1) > 0]
df_fin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1268770 entries, 1690 to 21096072
Data columns (total 27 columns):
6m                 1160726 non-null float64
5m                 1163250 non-null float64
4m                 1164737 non-null float64
3m                 1165245 non-null float64
2m                 1164501 non-null float64
1m                 1162572 non-null float64
3p                 1142170 non-null float64
2p                 1121981 non-null float64
1p                 1102031 non-null float64
6m_rel             708235 non-null float64
5m_rel             731534 non-null float64
4m_rel             746535 non-null float64
3m_rel             757651 non-null float64
2m_rel             767948 non-null float64
1m_rel             775509 non-null float64
3m_canc            52942 non-null float64
2m_canc            52503 non-null float64
1m_canc            52011 non-null float64
GENDER             1268261 non-null object
MARITAL_STATUS     1173881 non-null object
EDUCATION        

# Training: churn exacto 3 meses

In [76]:
df_fin["Churn"] = 0
#Regla churn stricta
df_fin.loc[ pd.isnull(df_fin["3p"]) &  pd.isnull(df_fin["2p"])  &  pd.isnull(df_fin["1p"]) &  pd.notnull(df_fin["1m"]) &  pd.notnull(df_fin["3m_canc"])  , "Churn"] = 1

In [77]:
df_fin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1268770 entries, 1690 to 21096072
Data columns (total 28 columns):
6m                 1160726 non-null float64
5m                 1163250 non-null float64
4m                 1164737 non-null float64
3m                 1165245 non-null float64
2m                 1164501 non-null float64
1m                 1162572 non-null float64
3p                 1142170 non-null float64
2p                 1121981 non-null float64
1p                 1102031 non-null float64
6m_rel             708235 non-null float64
5m_rel             731534 non-null float64
4m_rel             746535 non-null float64
3m_rel             757651 non-null float64
2m_rel             767948 non-null float64
1m_rel             775509 non-null float64
3m_canc            52942 non-null float64
2m_canc            52503 non-null float64
1m_canc            52011 non-null float64
GENDER             1268261 non-null object
MARITAL_STATUS     1173881 non-null object
EDUCATION        

In [78]:
df_fin["Churn"].value_counts()

0    1246806
1      21964
Name: Churn, dtype: int64

In [79]:
X = df_fin.drop(["3p", "2p", "1p", "3m_canc", "2m_canc", "1m_canc"], axis=1)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1268770 entries, 1690 to 21096072
Data columns (total 22 columns):
6m                 1160726 non-null float64
5m                 1163250 non-null float64
4m                 1164737 non-null float64
3m                 1165245 non-null float64
2m                 1164501 non-null float64
1m                 1162572 non-null float64
6m_rel             708235 non-null float64
5m_rel             731534 non-null float64
4m_rel             746535 non-null float64
3m_rel             757651 non-null float64
2m_rel             767948 non-null float64
1m_rel             775509 non-null float64
GENDER             1268261 non-null object
MARITAL_STATUS     1173881 non-null object
EDUCATION          436749 non-null object
EDUCATION_LEVEL    43629 non-null object
CITY               1051238 non-null object
LOCALITY           1051238 non-null object
edad               1258973 non-null float64
ModoPago           1268770 non-null object
edad_r             

In [80]:
#X.fillna(0, inplace=True) #reemplazo nulos por 0
#X = X.astype("uint8") #cambio el tipo de los datos
#X.info()

In [81]:
#X["id_cliente"] = X.index

In [82]:
#X.drop_duplicates(inplace=True)
#X.info()

In [83]:
X.to_csv("../../data/processed/4.4/train/m1.0 - mes actual - churn definido  - 6m.N.polizaXmes y Info.Cliente.M.Pago.csv")

# dummies

In [84]:
df = pd.read_csv("../../data/processed/4.4/train/m1.0 - mes actual - churn definido  - 6m.N.polizaXmes y Info.Cliente.M.Pago.csv", index_col="Unnamed: 0")
df.info()

/home/seba/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/seba/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1268770 entries, 1690 to 21096072
Data columns (total 22 columns):
6m                 1160726 non-null float64
5m                 1163250 non-null float64
4m                 1164737 non-null float64
3m                 1165245 non-null float64
2m                 1164501 non-null float64
1m                 1162572 non-null float64
6m_rel             708235 non-null float64
5m_rel             731534 non-null float64
4m_rel             746535 non-null float64
3m_rel             757651 non-null float64
2m_rel             767948 non-null float64
1m_rel             775509 non-null float64
GENDER             1268261 non-null object
MARITAL_STATUS     1173881 non-null object
EDUCATION          436749 non-null object
EDUCATION_LEVEL    43629 non-null object
CITY               1051238 non-null object
LOCALITY           1051238 non-null object
edad               1258973 non-null float64
ModoPago           1268770 non-null object
edad_r             

In [85]:
X = df.drop(['CITY','LOCALITY', 'edad'], axis=1)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1268770 entries, 1690 to 21096072
Data columns (total 19 columns):
6m                 1160726 non-null float64
5m                 1163250 non-null float64
4m                 1164737 non-null float64
3m                 1165245 non-null float64
2m                 1164501 non-null float64
1m                 1162572 non-null float64
6m_rel             708235 non-null float64
5m_rel             731534 non-null float64
4m_rel             746535 non-null float64
3m_rel             757651 non-null float64
2m_rel             767948 non-null float64
1m_rel             775509 non-null float64
GENDER             1268261 non-null object
MARITAL_STATUS     1173881 non-null object
EDUCATION          436749 non-null object
EDUCATION_LEVEL    43629 non-null object
ModoPago           1268770 non-null object
edad_r             1258973 non-null float64
Churn              1268770 non-null int64
dtypes: float64(13), int64(1), object(5)
memory usage: 193.6+ M

In [86]:
X = pd.get_dummies(X)
X.fillna(0, inplace=True) #reemplazo nulos por 0
X = X.astype("uint8") #cambio el tipo de los datos
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1268770 entries, 1690 to 21096072
Data columns (total 37 columns):
6m                               1268770 non-null uint8
5m                               1268770 non-null uint8
4m                               1268770 non-null uint8
3m                               1268770 non-null uint8
2m                               1268770 non-null uint8
1m                               1268770 non-null uint8
6m_rel                           1268770 non-null uint8
5m_rel                           1268770 non-null uint8
4m_rel                           1268770 non-null uint8
3m_rel                           1268770 non-null uint8
2m_rel                           1268770 non-null uint8
1m_rel                           1268770 non-null uint8
edad_r                           1268770 non-null uint8
Churn                            1268770 non-null uint8
GENDER_F                         1268770 non-null uint8
GENDER_M                         1268770 no

In [87]:
X.to_csv("../../data/processed/4.4/train/m1.0 - mes actual dumies - 6m.N.polizaXmes y Info.Cliente.M.Pago.csv")

In [88]:
X[X.iloc[:,:6].sum(axis=1) == 0]

,6m,5m,4m,3m,2m,1m,6m_rel,5m_rel,4m_rel,3m_rel,...,EDUCATION_LEVEL_SECUNDARIO,EDUCATION_LEVEL_TERCIARIO,EDUCATION_LEVEL_UNIVERSITARIO,ModoPago_BA,ModoPago_CC,ModoPago_Mix,ModoPago_PP,ModoPago_SJ,ModoPago_TA,ModoPago_TM


In [ ]:
#fin!!!!

In [14]:
df = pd.read_csv("../../data/processed/m1.0 - dummy sin churn definido  - 6m.N.polizaXmes y Info.Cliente.M.Pago.csv", index_col="Unnamed: 0")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 875419 entries, 1690 to 21260067
Data columns (total 59 columns):
6m                                875419 non-null int64
5m                                875419 non-null int64
4m                                875419 non-null int64
3m                                875419 non-null int64
2m                                875419 non-null int64
1m                                875419 non-null int64
3p                                875419 non-null int64
2p                                875419 non-null int64
1p                                875419 non-null int64
edad_r                            875419 non-null int64
GENDER_F                          875419 non-null int64
GENDER_M                          875419 non-null int64
MARITAL_STATUS_Casado             875419 non-null int64
MARITAL_STATUS_Conviv             875419 non-null int64
MARITAL_STATUS_Divorciado         875419 non-null int64
MARITAL_STATUS_No Corresp         875419 non

In [15]:
df["Churn"] = 0
#Regla churn stricta
#df.loc[ (df["3p"] == 0) & (df["2p"] == 0)  &  (df["1p"] == 0) &  (df["1m"] != 0) &  (df.iloc[:,:3].count(axis=1) > 0)  &  (df.iloc[:,1:4].count(axis=1) > 0) &  (df.iloc[:,2:5].count(axis=1) > 0), "Churn"] = 1
#Regla churn no stricta
df.loc[ (df["3p"] == 0) & (df["2p"] == 0)  &  (df["1p"] == 0) &  (df["1m"] != 0) , "Churn"] = 1

In [16]:
df.head()

,6m,5m,4m,3m,2m,1m,3p,2p,1p,edad_r,...,CITY_TUCUMAN,ModoPago_BA,ModoPago_CC,ModoPago_Mix,ModoPago_PP,ModoPago_SJ,ModoPago_TA,ModoPago_TM,id_cliente,Churn
1690,0,0,0,0,0,0,0,0,0,5,...,0,0,1,0,0,0,0,0,1690,0
1690,1,0,0,0,0,0,0,0,0,5,...,0,0,1,0,0,0,0,0,1690,0
1690,1,1,0,0,0,0,0,0,0,5,...,0,0,1,0,0,0,0,0,1690,0
1690,1,1,1,0,0,0,0,0,0,5,...,0,0,1,0,0,0,0,0,1690,0
1690,1,1,1,1,0,0,0,0,0,5,...,0,0,1,0,0,0,0,0,1690,0


In [18]:
df["Churn"].value_counts() / len(df)

0    0.969396
1    0.030604
Name: Churn, dtype: float64

In [89]:
df.drop("id_cliente", axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 875419 entries, 1690 to 21260067
Data columns (total 59 columns):
6m                                875419 non-null int64
5m                                875419 non-null int64
4m                                875419 non-null int64
3m                                875419 non-null int64
2m                                875419 non-null int64
1m                                875419 non-null int64
3p                                875419 non-null int64
2p                                875419 non-null int64
1p                                875419 non-null int64
edad_r                            875419 non-null int64
GENDER_F                          875419 non-null int64
GENDER_M                          875419 non-null int64
MARITAL_STATUS_Casado             875419 non-null int64
MARITAL_STATUS_Conviv             875419 non-null int64
MARITAL_STATUS_Divorciado         875419 non-null int64
MARITAL_STATUS_No Corresp         875419 non

In [90]:
df.drop(["3p", "2p", "1p"], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 875419 entries, 1690 to 21260067
Data columns (total 56 columns):
6m                                875419 non-null int64
5m                                875419 non-null int64
4m                                875419 non-null int64
3m                                875419 non-null int64
2m                                875419 non-null int64
1m                                875419 non-null int64
edad_r                            875419 non-null int64
GENDER_F                          875419 non-null int64
GENDER_M                          875419 non-null int64
MARITAL_STATUS_Casado             875419 non-null int64
MARITAL_STATUS_Conviv             875419 non-null int64
MARITAL_STATUS_Divorciado         875419 non-null int64
MARITAL_STATUS_No Corresp         875419 non-null int64
MARITAL_STATUS_Separado           875419 non-null int64
MARITAL_STATUS_Soltero            875419 non-null int64
MARITAL_STATUS_Union Civ          875419 non

In [91]:
df.to_csv("../../data/processed/train/m1.0 - churn estricto - 6m.N.polizaXmes y Info.Cliente.M.Pago.csv")